# VWAP & HMA Crossover Strategy - Backtesting Analysis

This notebook demonstrates how to backtest the VWAP & HMA Crossover intraday trading strategy on historical forex data.

## Strategy Overview

The VWAP & HMA Crossover strategy is a trend-following intraday strategy that combines institutional volume bias (VWAP) with momentum signals (HMA crossovers).

### Key Concepts
- **VWAP (Volume Weighted Average Price)**: Represents the average price weighted by volume. Resets daily. Acts as the "fair value" benchmark used by institutional traders.
- **HMA (Hull Moving Average)**: A fast-reacting moving average that reduces lag while maintaining smoothness. More responsive than traditional MAs.

### Entry Rules
- **LONG**: Price > VWAP (bullish bias) AND HMA crosses above price (bullish momentum)
- **SHORT**: Price < VWAP (bearish bias) AND HMA crosses below price (bearish momentum)

### Exit Rules
- **Stop Loss**: 1.5× ATR(14) from entry price
- **Take Profit**: 2× stop loss distance (1:2 R:R)
- **Time Exit**: Close all positions 30 minutes before market close (avoid overnight fees)

### Expected Performance
- Win Rate: 50-60%
- Risk/Reward: 1:2
- Trades per day: 3-5
- Best pairs: EUR_USD, GBP_USD

### Files
- Strategy implementation: `src/strategies/vwap_hma_crossover.py`
- Backtest notebook: `notebooks/03_vwap_hma_crossover_backtest.ipynb`

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest
from pathlib import Path

# Import strategy classes
from src.strategies.vwap_hma_crossover import VWAPHMACrossover, VWAPHMACrossoverOptimized

# Import data fetching classes
from src.oanda_client import OandaClient
from src.data_retriever import HistoricalDataRetriever
from src.data_storage import DataStorage

# Import pandas_ta for indicator calculations
import pandas_ta as ta

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Imports successful")

## 2. Configuration

Set the parameters for our backtest.

In [ ]:
# Data parameters
INSTRUMENT = 'EUR_USD'
GRANULARITY = 'M15'  # 15-minute candles for intraday trading
FROM_DATE = '20250101'
TO_DATE = '20251231'

# Backtest parameters
INITIAL_CASH = 10000
COMMISSION = 0.0001  # 1 pip for forex

# File paths
DATA_DIR = Path('../data/historical')
DATA_PATH = DATA_DIR / INSTRUMENT / f'{INSTRUMENT}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'

# API configuration path
CONFIG_PATH = Path('../config/oanda_config.ini')

print(f"Configuration:")
print(f"  Instrument: {INSTRUMENT}")
print(f"  Timeframe: {GRANULARITY} (intraday)")
print(f"  Period: {FROM_DATE} to {TO_DATE}")
print(f"  Initial Capital: ${INITIAL_CASH:,.2f}")
print(f"  Commission: {COMMISSION*100:.2f}%")
print(f"  Data Path: {DATA_PATH}")
print(f"  Data Exists: {DATA_PATH.exists()}")

## 3. Load Historical Data

Load M15 data with volume for VWAP calculation.

In [ ]:
def fetch_and_save_data(instrument: str, granularity: str, from_date: str, to_date: str) -> pd.DataFrame:
    """
    Fetch historical data from OANDA API and save to CSV.

    Args:
        instrument: Currency pair (e.g., 'EUR_USD')
        granularity: Timeframe (e.g., 'M15')
        from_date: Start date in YYYYMMDD format
        to_date: End date in YYYYMMDD format

    Returns:
        DataFrame with OHLCV data
    """
    print(f"Fetching {instrument} {granularity} data from OANDA API...")
    print(f"Date range: {from_date} to {to_date}")

    # Convert YYYYMMDD to YYYY-MM-DD for API
    from_date_api = f"{from_date[:4]}-{from_date[4:6]}-{from_date[6:8]}"
    to_date_api = f"{to_date[:4]}-{to_date[4:6]}-{to_date[6:8]}"

    # Initialize OANDA client
    client = OandaClient(environment='practice', config_path=str(CONFIG_PATH))
    print("✓ OANDA client initialized")

    # Initialize data retriever
    retriever = HistoricalDataRetriever(client)

    # Fetch data
    df = retriever.fetch_historical_data(
        instrument=instrument,
        granularity=granularity,
        from_date=from_date_api,
        to_date=to_date_api
    )

    if df.empty:
        raise ValueError(f"No data retrieved for {instrument}")

    print(f"✓ Retrieved {len(df):,} candles")

    # Save to CSV
    storage = DataStorage(base_path=str(DATA_DIR))
    file_path = storage.save_to_csv(
        df=df,
        instrument=instrument,
        granularity=granularity,
        from_date=from_date_api,
        to_date=to_date_api
    )
    print(f"✓ Saved to: {file_path}")

    return df


def load_data(data_path: Path) -> pd.DataFrame:
    """
    Load data from CSV file.

    Args:
        data_path: Path to CSV file

    Returns:
        DataFrame formatted for backtesting.py
    """
    print(f"Loading data from: {data_path}")

    # Read CSV, skipping header comments
    df = pd.read_csv(data_path, comment='#', parse_dates=['time'], index_col='time')

    # Rename columns to match backtesting.py expectations
    df.columns = [col.capitalize() for col in df.columns]

    # Keep only required columns
    required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[col for col in required_cols if col in df.columns]]

    return df


def ensure_data_exists(data_path: Path, instrument: str, granularity: str, from_date: str, to_date: str) -> pd.DataFrame:
    """
    Ensure data exists, fetch if missing.
    """
    if data_path.exists():
        print(f"✓ Data file found: {data_path}")
        return load_data(data_path)
    else:
        print(f"✗ Data file not found: {data_path}")
        print(f"  Downloading from OANDA API...\n")
        fetch_and_save_data(instrument, granularity, from_date, to_date)
        return load_data(data_path)


# Load data
print("="*60)
print("LOADING HISTORICAL DATA")
print("="*60)

df = ensure_data_exists(DATA_PATH, INSTRUMENT, GRANULARITY, FROM_DATE, TO_DATE)

print(f"\n✓ Loaded {len(df):,} candles")
print(f"  Date range: {df.index.min()} to {df.index.max()}")
print(f"  Duration: {(df.index.max() - df.index.min()).days} days")

print(f"\nData sample:")
display(df.head())

print(f"\nData statistics:")
display(df.describe())

## 4. Indicator Visualization

Calculate and visualize VWAP, HMA, and ATR indicators.

In [ ]:
# Calculate indicators for visualization
typical_price = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate VWAP (simplified - resets daily)
df['date'] = df.index.date
df['TypicalPrice'] = typical_price
df['TP_x_Vol'] = df['TypicalPrice'] * df['Volume']

# Group by date and calculate cumulative sums
df['VWAP'] = df.groupby('date').apply(
    lambda x: (x['TP_x_Vol'].cumsum() / x['Volume'].cumsum())
).reset_index(level=0, drop=True)

# Calculate HMA
df['HMA'] = ta.hma(df['Close'], length=21)

# Calculate ATR
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

print("✓ Indicators calculated")
print(f"\nIndicator preview:")
display(df[['Close', 'VWAP', 'HMA', 'ATR']].tail(10))

In [ ]:
# Visualize indicators on a subset of data (last 200 bars)
plot_df = df.iloc[-200:].copy()

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('Price, VWAP & HMA', 'ATR'),
    row_heights=[0.7, 0.3]
)

# Price candlestick
fig.add_trace(
    go.Candlestick(
        x=plot_df.index,
        open=plot_df['Open'],
        high=plot_df['High'],
        low=plot_df['Low'],
        close=plot_df['Close'],
        name='Price'
    ),
    row=1, col=1
)

# VWAP line
fig.add_trace(
    go.Scatter(
        x=plot_df.index,
        y=plot_df['VWAP'],
        name='VWAP',
        line=dict(color='blue', width=2)
    ),
    row=1, col=1
)

# HMA line
fig.add_trace(
    go.Scatter(
        x=plot_df.index,
        y=plot_df['HMA'],
        name='HMA(21)',
        line=dict(color='orange', width=2)
    ),
    row=1, col=1
)

# ATR
fig.add_trace(
    go.Scatter(
        x=plot_df.index,
        y=plot_df['ATR'],
        name='ATR(14)',
        line=dict(color='purple', width=2)
    ),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="ATR", row=2, col=1)

fig.update_layout(
    height=800,
    title_text=f"{INSTRUMENT} {GRANULARITY} - VWAP & HMA Indicators (Last 200 bars)",
    xaxis_rangeslider_visible=False
)

fig.show()

## 5. Signal Frequency Analysis

Analyze how often entry conditions are met.

In [ ]:
print("="*60)
print("SIGNAL FREQUENCY ANALYSIS")
print("="*60)

# Calculate signal conditions
# VWAP bias
above_vwap = df['Close'] > df['VWAP']
below_vwap = df['Close'] < df['VWAP']

# HMA crossovers
hma_above_price = (df['HMA'] > df['Close']) & (df['HMA'].shift(1) <= df['Close'].shift(1))
hma_below_price = (df['HMA'] < df['Close']) & (df['HMA'].shift(1) >= df['Close'].shift(1))

# Combined signals
long_signals = above_vwap & hma_above_price
short_signals = below_vwap & hma_below_price

print(f"\nVWAP Bias:")
print(f"  Price > VWAP (bullish): {above_vwap.sum():,} bars ({above_vwap.sum()/len(df)*100:.1f}%)")
print(f"  Price < VWAP (bearish): {below_vwap.sum():,} bars ({below_vwap.sum()/len(df)*100:.1f}%)")

print(f"\nHMA Crossovers:")
print(f"  HMA crosses above price: {hma_above_price.sum():,} signals")
print(f"  HMA crosses below price: {hma_below_price.sum():,} signals")

print(f"\nCombined Entry Signals:")
print(f"  LONG signals: {long_signals.sum():,}")
print(f"  SHORT signals: {short_signals.sum():,}")
print(f"  Total signals: {long_signals.sum() + short_signals.sum():,}")
print(f"  Expected trades per day: {(long_signals.sum() + short_signals.sum()) / ((df.index.max() - df.index.min()).days):.1f}")

## 6. Run Backtest

Run the VWAP & HMA Crossover strategy backtest.

In [ ]:
# Prepare data for backtest (remove temporary columns)
df_backtest = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

# Initialize backtest
bt = Backtest(df_backtest, VWAPHMACrossover, cash=INITIAL_CASH, commission=COMMISSION)

# Run backtest
print("Running backtest...\n")
stats = bt.run()

print("="*80)
print("BACKTEST RESULTS - VWAP & HMA CROSSOVER STRATEGY")
print("="*80)
print(stats)
print("="*80)

### Key Metrics Explanation

- **Return [%]**: Total percentage return over the period
- **Sharpe Ratio**: Risk-adjusted return (> 1.0 is good, > 2.0 is excellent)
- **Max Drawdown [%]**: Largest peak-to-trough decline
- **Win Rate [%]**: Percentage of profitable trades
- **Profit Factor**: Gross profit / Gross loss (> 1.0 is profitable)
- **Exposure Time [%]**: Percentage of time in position

## 7. Equity Curve Visualization

In [ ]:
# Extract equity curve
equity = stats['_equity_curve']

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Equity Curve', 'Drawdown'),
    row_heights=[0.7, 0.3]
)

# Equity curve
fig.add_trace(
    go.Scatter(x=equity.index, y=equity['Equity'], name='Equity', line=dict(color='blue', width=2)),
    row=1, col=1
)

# Add starting capital line
fig.add_hline(y=INITIAL_CASH, line_dash="dash", line_color="gray", annotation_text="Initial Capital", row=1, col=1)

# Drawdown
fig.add_trace(
    go.Scatter(x=equity.index, y=equity['DrawdownPct'], name='Drawdown',
               fill='tozeroy', line=dict(color='red', width=1)),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Equity ($)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

fig.update_layout(height=800, title_text="Strategy Performance - VWAP & HMA Crossover", showlegend=True)
fig.show()

## 8. Trade Analysis

In [ ]:
# Extract trades
trades = stats['_trades']

if len(trades) > 0:
    print(f"Total Trades: {len(trades)}")
    print(f"\nTrade Summary:")
    display(trades)

    # Calculate additional metrics
    winning_trades = trades[trades['PnL'] > 0]
    losing_trades = trades[trades['PnL'] < 0]

    print(f"\n{'='*60}")
    print("TRADE BREAKDOWN")
    print("="*60)
    print(f"Winning trades: {len(winning_trades)} ({len(winning_trades)/len(trades)*100:.1f}%)")
    print(f"Losing trades: {len(losing_trades)} ({len(losing_trades)/len(trades)*100:.1f}%)")
    print(f"\nLong trades: {len(trades[trades['Size'] > 0])} ({len(trades[trades['Size'] > 0])/len(trades)*100:.1f}%)")
    print(f"Short trades: {len(trades[trades['Size'] < 0])} ({len(trades[trades['Size'] < 0])/len(trades)*100:.1f}%)")

    if len(winning_trades) > 0:
        print(f"\nAverage winning trade: ${winning_trades['PnL'].mean():.2f}")
        print(f"Best trade: ${winning_trades['PnL'].max():.2f}")

    if len(losing_trades) > 0:
        print(f"\nAverage losing trade: ${losing_trades['PnL'].mean():.2f}")
        print(f"Worst trade: ${losing_trades['PnL'].min():.2f}")
else:
    print("⚠️ No trades executed. Strategy conditions may be too strict.")

### Trade Distribution Visualization

In [ ]:
if len(trades) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Trade returns histogram
    axes[0, 0].hist(trades['ReturnPct'] * 100, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=1)
    axes[0, 0].set_xlabel('Return (%)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Trade Return Distribution')
    axes[0, 0].grid(alpha=0.3)

    # PnL histogram
    axes[0, 1].hist(trades['PnL'], bins=20, edgecolor='black', alpha=0.7, color='green')
    axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=1)
    axes[0, 1].set_xlabel('PnL ($)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Trade PnL Distribution')
    axes[0, 1].grid(alpha=0.3)

    # Cumulative PnL
    axes[1, 0].plot(range(len(trades)), trades['PnL'].cumsum(), marker='o', markersize=3, linewidth=1.5)
    axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=1)
    axes[1, 0].set_xlabel('Trade Number')
    axes[1, 0].set_ylabel('Cumulative PnL ($)')
    axes[1, 0].set_title('Cumulative PnL Over Trades')
    axes[1, 0].grid(alpha=0.3)

    # Trade duration
    trade_durations = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds() / 3600  # in hours
    axes[1, 1].hist(trade_durations, bins=20, edgecolor='black', alpha=0.7, color='orange')
    axes[1, 1].set_xlabel('Duration (hours)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Trade Duration Distribution')
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nTrade Duration Statistics:")
    print(f"  Average duration: {trade_durations.mean():.2f} hours")
    print(f"  Median duration: {trade_durations.median():.2f} hours")
    print(f"  Min duration: {trade_durations.min():.2f} hours")
    print(f"  Max duration: {trade_durations.max():.2f} hours")
else:
    print("No trades to visualize.")

## 9. Visualize Entry Signals

Plot price with VWAP, HMA, and entry signals on a subset of data.

In [ ]:
if len(trades) > 0:
    # Get trades from a specific period (e.g., first 30 days)
    plot_start = df.index.min()
    plot_end = plot_start + pd.Timedelta(days=30)
    plot_df = df[(df.index >= plot_start) & (df.index <= plot_end)].copy()
    plot_trades = trades[(trades['EntryTime'] >= plot_start) & (trades['EntryTime'] <= plot_end)]

    if len(plot_df) > 0 and len(plot_trades) > 0:
        fig = go.Figure()

        # Candlestick
        fig.add_trace(go.Candlestick(
            x=plot_df.index,
            open=plot_df['Open'],
            high=plot_df['High'],
            low=plot_df['Low'],
            close=plot_df['Close'],
            name='Price'
        ))

        # VWAP
        fig.add_trace(go.Scatter(
            x=plot_df.index,
            y=plot_df['VWAP'],
            name='VWAP',
            line=dict(color='blue', width=2, dash='dot')
        ))

        # HMA
        fig.add_trace(go.Scatter(
            x=plot_df.index,
            y=plot_df['HMA'],
            name='HMA(21)',
            line=dict(color='orange', width=2)
        ))

        # Entry signals
        long_trades = plot_trades[plot_trades['Size'] > 0]
        short_trades = plot_trades[plot_trades['Size'] < 0]

        if len(long_trades) > 0:
            fig.add_trace(go.Scatter(
                x=long_trades['EntryTime'],
                y=long_trades['EntryPrice'],
                mode='markers',
                name='Long Entry',
                marker=dict(symbol='triangle-up', size=15, color='green')
            ))

        if len(short_trades) > 0:
            fig.add_trace(go.Scatter(
                x=short_trades['EntryTime'],
                y=short_trades['EntryPrice'],
                mode='markers',
                name='Short Entry',
                marker=dict(symbol='triangle-down', size=15, color='red')
            ))

        fig.update_layout(
            title=f"{INSTRUMENT} - Entry Signals (First 30 Days)",
            xaxis_title="Date",
            yaxis_title="Price",
            height=700,
            xaxis_rangeslider_visible=False
        )

        fig.show()
    else:
        print("No trades in the selected period.")
else:
    print("No trades to visualize.")

## 10. Parameter Optimization (Optional)

Optimize strategy parameters to find better settings.

In [ ]:
# Optimize HMA period and ATR multiplier
print("Running parameter optimization...\n")
print("This may take several minutes...\n")

optimization_stats = bt.optimize(
    hma_period=range(15, 31, 5),
    atr_multiplier=[1.0, 1.5, 2.0, 2.5],
    risk_reward=[1.5, 2.0, 2.5, 3.0],
    maximize='Sharpe Ratio',
    constraint=lambda p: p.hma_period > 0
)

print("="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(optimization_stats)
print("="*80)

print(f"\nBest parameters:")
print(f"  HMA Period: {optimization_stats._strategy.hma_period}")
print(f"  ATR Multiplier: {optimization_stats._strategy.atr_multiplier}")
print(f"  Risk/Reward: {optimization_stats._strategy.risk_reward}")

## 11. Summary and Conclusions

### Strategy Performance
Review the backtest results above and compare against expected performance:

**Expected:**
- Win Rate: 50-60%
- Risk/Reward: 1:2
- Trades per day: 3-5
- Max Drawdown: <15%
- Sharpe Ratio: >1.0

### Key Insights
1. **VWAP as trend filter**: Check if price > VWAP correlates with winning long trades
2. **HMA crossover timing**: Evaluate entry timing quality and false signals
3. **Risk management**: Verify 1:2 R/R is being achieved on average
4. **Intraday focus**: Confirm no overnight positions (all trades close same day)

### Next Steps
- Test on different instruments (GBP_USD, USD_JPY)
- Test on different time periods (different market conditions)
- Consider adding volume filter (VWAPHMACrossoverOptimized)
- Consider adding session filters (London/NY overlap)
- Forward test on demo account before live trading

### Notes
- This strategy works best in trending markets
- May underperform in ranging/choppy conditions
- Requires clean volume data for accurate VWAP calculation
- Consider market hours and liquidity when trading live